# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846834837157                   -0.70    4.8
  2   -7.852319554563       -2.26       -1.53    1.0
  3   -7.852646164816       -3.49       -2.52    3.2
  4   -7.852646676595       -6.29       -3.36    2.2
  5   -7.852646685893       -8.03       -4.73    1.5
  6   -7.852646686724       -9.08       -5.27    3.8
  7   -7.852646686730      -11.27       -6.08    1.2
  8   -7.852646686730      -12.56       -7.58    2.2
  9   -7.852646686730      -14.35       -7.57    2.5
 10   -7.852646686730      -15.05       -8.76    2.0
 11   -7.852646686730   +  -15.05       -9.58    2.5
 12   -7.852646686730   +  -14.75       -9.83    1.0
 13   -7.852646686730      -15.05      -11.21    2.0
 14   -7.852646686730      -15.05      -11.42    1.8
 15   -7.852646686730   +  -14.75      -12.46    2.5


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846854167540                   -0.70           4.8
  2   -7.852528555528       -2.25       -1.63   0.80    2.0
  3   -7.852637021499       -3.96       -2.71   0.80    1.0
  4   -7.852646500247       -5.02       -3.28   0.80    2.2
  5   -7.852646681902       -6.74       -4.11   0.80    1.2
  6   -7.852646686339       -8.35       -4.81   0.80    1.2
  7   -7.852646686721       -9.42       -5.79   0.80    1.8
  8   -7.852646686730      -11.07       -6.62   0.80    2.2
  9   -7.852646686730      -12.88       -7.77   0.80    2.0
 10   -7.852646686730      -15.05       -8.38   0.80    2.2
 11   -7.852646686730      -15.05       -8.47   0.80    2.0
 12   -7.852646686730   +    -Inf       -9.50   0.80    1.0
 13   -7.852646686730      -14.75       -9.99   0.80    2.0
 14   -7.852646686730   +  -14.75      -10.82   0.80    1.2
 15   -7.852646686730      -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.393057e+01     3.733603e+00
 * time: 0.4572129249572754
     1     1.024307e+00     1.950170e+00
 * time: 0.6680440902709961
     2    -1.490782e+00     2.246964e+00
 * time: 0.6949639320373535
     3    -3.814900e+00     1.825841e+00
 * time: 0.8629879951477051
     4    -4.966042e+00     1.656600e+00
 * time: 0.9015960693359375
     5    -6.777482e+00     8.934188e-01
 * time: 0.9403579235076904
     6    -7.395787e+00     5.009778e-01
 * time: 0.9789650440216064
     7    -7.632528e+00     2.711337e-01
 * time: 1.0057039260864258
     8    -7.726262e+00     8.520718e-02
 * time: 1.0324039459228516
     9    -7.759652e+00     1.241048e-01
 * time: 1.0589759349822998
    10    -7.775273e+00     6.105748e-02
 * time: 1.0857460498809814
    11    -7.782735e+00     7.089747e-02
 * time: 1.1125049591064453
    12    -7.786939e+00     6.312374e-02
 * time: 1.1389639377593994
    13    -7.795833e+00     6.603219e-02
 * time: 1.1654949188

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846791552104                   -0.70    4.8
  2   -7.852312649078       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686698                   -2.55
  2   -7.852646686730      -10.49       -5.86
  3   -7.852646686730   +  -14.35      -12.59


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 8.469304733629998e-14
|ρ_newton - ρ_scfv| = 8.27566518069133e-13
|ρ_newton - ρ_dm|   = 1.1382784197640866e-10
